# I have not really had much time to clean up the code

But what I can do is try to walk you through how to use this thing. incidentally this is also the demo code I used for my project presentation.

## 1. Program a drawing

Essentially, you use your mouse to trace over you favourite drawing, or a blank canvas.
Hold down the 'x' key and move the mouse to make your stroke. Release it to stop the stroke. Press it down again to create another stroke. Close the window when you're done.

In [82]:
import drawing
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import pickle

# Uncomment these to use PIL to open your favourite picture in the pics folder
#img = Image.open("jo_mama.jpeg")  
#img = np.asarray(img)[:,:,0]

img = np.zeros([600,600])  # or just use a blank canvas

%matplotlib qt # You want to use the matplotlib GTK pyQT GUI for this

d = drawing.Drawing(img)  # This is how you make your drawing!
strokes = d.get_strokes() # get out whatever you did here

xs, ys = strokes[0], strokes[1]

%matplotlib inline  # Ayy go back to plotting stuff in the notebook
for a,b in zip(xs,ys):   # maybe you want to see what you did?
    plt.plot(a,b, "r-")

strokes = []
scale = max(d.get_dims()) # here I am scaling this using the data in the calibration.py file, and my 
                          # own preferences of fitting my demo drawing onto a sticky note sized area
for a,b in zip(xs,ys):
    a = 7*(a/scale)+9     # xmin = 9,    xmax = 9+7
    b = -7*(b/scale)-5    # ymin = -5-7, xmax = -5
    strokes.append((a,b))

for (a,b) in strokes:   # maybe you want to see your scaled image
    plt.plot(a,b,"r-")

pickle.dump(strokes, open( "your_filename_here.p", "wb" ) )  # Save it as a pickle for later use! why draw
                                                             # again when you can save your masterpiece?

## Initialise picosso

Import all the stuff you need, and initiate serial communication

In [15]:
import serial
from time import sleep
from calibration import scale_angles, l1, l2
from inv_kinematics import InvKinematics
import numpy as np
import pickle


port = "/dev/ttyACM3" # this is my port on a linux machine. you may need to change 
                      # this depending on your platform
robot = serial.Serial(port, 9600) 
k = InvKinematics(l1,l2)      # Define the robot's arm lengths
robot.write(f"r\n".encode())  # Raise the pen
sleep(1)                      # Wait, let thing settle  
robot.reset_input_buffer()    # purge the serial buffers, or the robot freezes
robot.reset_output_buffer()

## Draw

In [18]:
# Remember the pickled drawings?
# just load one of those!
drawing = "ein"
#drawing = "birb"
#drawing = "doge"
#drawing = "flamingo"

strokes = pickle.load(open(f"drawings/{drawing}.p", 'rb'))

i = 0 # lets us know how many serial commands have been sent. 
      # serial budffers need to be purged regularly to prevent freezing
for (a,b) in strokes:
    new = True # flag for new stroke just activated
    for x, y in zip(a,b):
        a1, a2 = k.invert(x, y)          # invert to ideal real world angles
        a1, a2 = scale_angles(a1, a2)    # scale to corresponding servo angles
        robot.write(f"a{a1}\n".encode()) # send the robot the angles
        robot.write(f"b{a2}\n".encode())
        if new:         # move pen while its raised
            sleep(0.5)  # wait for vibrations of arm to die out, since this is a new stroke
            new = False 
            robot.write("l\n".encode())
            sleep(0.5)  # lower the pen, and chill. you dont want erratic arcs on the paper.
        i += 1      # increment accumulator
        if i >= 10: # and clear buffers
            robot.reset_input_buffer()
            robot.reset_output_buffer()
            i = 0
        sleep(0.05) # avoid overloading the serial. Chill a bit
    robot.reset_input_buffer()   # after each stroke, clear buffers. Why not?
    robot.reset_output_buffer()
    robot.write("r\n".encode())  # Stroke ended. Raise the pen!


robot.write(f"a{180}\n".encode()) # the drawing is done! return robot to neutral position
robot.write(f"b{180}\n".encode()) # and unveil your drawing!

5